# Ceneo Scraper

## Lista kroków "manualnych"

1. Wejdź na stronę produktu / z opiniami o produkcie
2. Dla każdej opini na stronie 
    1. skopiuj opinię
    2. wklej opinię do edytora tekstu
3. Przejdź do kolejnej strony z opiniami
4. Powtarzaj kroki 2-3 dopóki są strony z opiniami 

## Lista krokow Scrapera

1. Wysłanie żądania dostępu do zasobu - strona z opiniami o produkcie 
2. Wydobycie z kodu HTML fragmentów odpowiadających opiniom
3. Dla każdej opinii na stronie
    1. Wydobycie z kodu opini jej składowych





## Struktura opini w serwisie Ceneo.pl

| składowa | nazwa | selektor |
|----------|-------|----------|
| identyfikator opinii|opinion_id|div.user-post user-post__card js_product-review|
| autor|author|span.user-post__author-name|
| rekomendacja|recomendation|span.user-post__author-recomendation|
| treść opinii|content|div.user-post__text|
| lista zalet|pros|div.review-feature > div:nth-child(1)|
| lista wad|cons|div.review-feature > div:nth-child(2)|
| data wystawienia opinii|post_date|span.user-post__published > time:nth-child(1)|
| data zakupu produktu|purchase_date|span.user-post__published > time:nth-child(2)|
| ile osób uznało opinię za przydatną|useful|button.vote-yes.js_product-review-vote.js_vote-yes"|
| ile osób uznało opinię za nieprzydatną|useless|button.vote-no.js_product-review-vote.js_vote-no > span|

(div.js_product-review)- przykład do selektora
znaleźć na ceneo przy jakieś opini 



63717975
124893467
1084811121
114228736
28020568

1. Import bibliotek

In [2]:
import requests
from bs4 import BeautifulSoup

2. Wysłanie żądania dostępu do zasobu (request)

In [4]:
product_code= "28020568"
url= f"https://www.ceneo.pl/{product_code}#tab=reviews"
response = requests.get(url)

200


3. Wydobycie z kodu HTML fragmentów odpowiadających opiniom

In [22]:
page = BeautifulSoup(response.text, "html.parser")
opinions = page.select("div.js_product-review")

<class 'bs4.element.ResultSet'>
<class 'bs4.element.Tag'>


4. Wydobycie z kodu pini jej składowych dla każdej opinii na stronie

In [ ]:
for opinion in opinions:
    pass